In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter
from scipy import stats
import seaborn as sns
from scipy.linalg import sqrtm
from sklearn.metrics import mean_squared_error
from math import sqrt

In [2]:
reviews = pd.read_csv('reviews_en_polarity.csv')
reviews.head()

/Applications/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (0,1) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,listing_id,id,date,reviewer_id,reviewer_name,comments,language,polarity
0,2384,25218143,2015-01-09,14385014.0,Ivan,it's a wonderful trip experience. I didn't exc...,en,0.9489
1,2384,28475392,2015-03-24,16241178.0,Namhaitou,This is my first trip using Airbnb. I was a li...,en,0.9519
2,189821,536083,2011-09-16,124036.0,Jenny,Meighan was a great host! Communication was e...,en,0.9665
3,189821,550149,2011-09-20,911939.0,Bonnie,"Great apartment, wonderful, helpful host! It ...",en,0.9458
4,189821,577757,2011-09-28,938072.0,Lauren,Amazing apt. Very clean and comfortable. Would...,en,0.9407


In [3]:
reviews.shape

(330603, 8)

In [4]:
df_rec = reviews[['reviewer_id', 'listing_id', 'polarity']].copy(deep=True)
df_rec.nunique()

reviewer_id    296653
listing_id       6800
polarity         8741
dtype: int64

In [5]:
df_rec = df_rec.dropna()

In [6]:
df_rec['u_id'] = df_rec['reviewer_id']
df_rec['i_id'] = df_rec['listing_id']
df_rec['rating'] = df_rec['polarity']
df_rec['u_id'] = df_rec['u_id'].astype('int')
df_rec['i_id'] = df_rec['i_id'].astype('int')

In [7]:
df_rec = df_rec[["u_id", "i_id", "rating"]]
df_rec

,u_id,i_id,rating
0,14385014,2384,0.9489
1,16241178,2384,0.9519
2,124036,189821,0.9665
3,911939,189821,0.9458
4,938072,189821,0.9407
...,...,...,...
330598,476732857,700756881572123144,0.8605
330599,375433341,700844721317041969,0.6369
330600,244580123,700938257605355958,0.9678
330601,104702793,701009010450223333,0.6800


In [25]:
df_rec.to_csv("rating.csv")

# Use SVD for Prediction

In [8]:
pip install git+https://github.com/gbolmier/funk-svd

  Cloning https://github.com/gbolmier/funk-svd to /private/var/folders/ct/8hnfxbqj6sj0_7lqw56c0c0c0000gn/T/pip-req-build-hciu_fh7
  Running command git clone --filter=blob:none --quiet https://github.com/gbolmier/funk-svd /private/var/folders/ct/8hnfxbqj6sj0_7lqw56c0c0c0000gn/T/pip-req-build-hciu_fh7
  Resolved https://github.com/gbolmier/funk-svd to commit fecc38ea1c2859ef6a6d9af0b7f953e1b693764e
  Preparing metadata (setup.py) ... done
You should consider upgrading via the '/Applications/anaconda3/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [9]:
from funk_svd.dataset import fetch_ml_ratings
from funk_svd import SVD
from sklearn.metrics import mean_absolute_error

In [10]:
train = df_rec.sample(frac=0.8, random_state=7)
val = df_rec.drop(train.index.tolist()).sample(frac=0.5, random_state=8)
test = df_rec.drop(train.index.tolist()).drop(val.index.tolist())

In [11]:
svd = SVD(lr=0.001, reg=0.005, n_epochs=100, n_factors=15,
          early_stopping=True, shuffle=False, min_rating=-1, max_rating=1)

In [12]:
svd.fit(X=train, X_val=val)

Preprocessing data...

Preprocessing data...

Epoch 1/100  | val_loss: 0.06 - val_rmse: 0.24 - val_mae: 0.15 - took 0.8 sec
Epoch 2/100  | val_loss: 0.06 - val_rmse: 0.24 - val_mae: 0.15 - took 0.0 sec
Epoch 3/100  | val_loss: 0.06 - val_rmse: 0.24 - val_mae: 0.15 - took 0.0 sec

Training took 2 sec


In [13]:
pred = svd.predict(test)

In [14]:
mae = mean_absolute_error(test['rating'], pred)

In [15]:
print(f'Test MAE: {mae:.2f}')

Test MAE: 0.15


In [16]:
len(pred)

32890

In [17]:
test['pred_rating'] = pred

In [18]:
test

,u_id,i_id,rating,pred_rating
14,1472130,189821,0.9689,0.891486
16,485527,189821,0.9877,0.962647
19,1497061,189821,0.8627,0.891486
22,2873370,2384,0.9809,0.860401
43,28889700,2384,0.9412,0.916040
...,...,...,...,...
330534,68272318,694935619325721273,0.9176,0.822035
330553,116263256,696244611219279084,0.7074,0.822830
330566,11754573,706570652065975692,0.9136,0.822027
330573,261000069,697879132687950994,0.9133,0.821648


In [19]:
test.sort_values(by=['pred_rating'], ascending=False, inplace=True)

In [20]:
test

,u_id,i_id,rating,pred_rating
280582,50913281,44552556,0.6369,0.976259
139197,24493789,21169118,0.9468,0.968359
305726,219742843,50787434,0.6249,0.967840
145806,110720891,20649033,0.6369,0.966699
162321,30123260,24288027,0.9715,0.964649
...,...,...,...,...
206384,190651298,29093384,0.9231,0.651288
207458,456726994,29093384,0.7845,0.646732
89760,124923168,13635380,-0.2755,0.622893
90206,216603448,13635380,0.8955,0.613426


# Recommend

In [21]:
df_rec.u_id.unique()

array([ 14385014,  16241178,    124036, ..., 375433341, 104702793,
       441060413])

In [22]:
list(df_rec.i_id.unique())

[2384,
 189821,
 7126,
 10945,
 12140,
 24833,
 25879,
 207218,
 28749,
 37738,
 207351,
 220333,
 225314,
 239140,
 250354,
 258253,
 67940,
 264547,
 71930,
 289689,
 289884,
 84042,
 94450,
 145659,
 290155,
 1356190,
 312192,
 345075,
 1361421,
 349265,
 350347,
 1392594,
 1461451,
 1502674,
 1502700,
 1554433,
 1573401,
 372929,
 404792,
 1584422,
 1591370,
 450648,
 464581,
 1614802,
 1620098,
 1623463,
 1648786,
 1663673,
 484898,
 507517,
 1668481,
 1678422,
 1721045,
 532862,
 542067,
 1732652,
 1759266,
 1759389,
 1769731,
 1772920,
 1773021,
 548905,
 1773025,
 1810118,
 584590,
 585353,
 1837153,
 585356,
 1854000,
 589939,
 628149,
 1921670,
 634310,
 640841,
 1944802,
 2011301,
 651742,
 662034,
 668262,
 697634,
 2062638,
 718224,
 2103986,
 2139407,
 724895,
 2189522,
 2208845,
 726376,
 2298310,
 2305194,
 2305230,
 727847,
 739495,
 2320027,
 755701,
 780063,
 790934,
 2396340,
 805890,
 805936,
 863120,
 872247,
 2437230,
 887038,
 2459132,
 903996,
 2471876,
 247917

In [23]:

def get_recommendations(df, u_id, N):
    i_ids = list(df.i_id.unique())
    fm = {'u_id': [u_id for i in range(len(i_ids))], 'i_id': i_ids}
    pred = pd.DataFrame.from_dict(fm)
    res = svd.predict(pred)
    pred['pred_rating'] = res
    pred.sort_values(by=['pred_rating'], ascending=False, inplace=True)
    return pred.head(N)
    

In [24]:

get_recommendations(df_rec, df_rec.sample().u_id.iloc[0], 10)

,u_id,i_id,pred_rating
1,85624211,189821,0.891486
395,85624211,8706915,0.885589
327,85624211,6065318,0.878450
225,85624211,5074013,0.876380
25,85624211,1356190,0.874814
9,85624211,37738,0.874684
108,85624211,929914,0.873821
54,85624211,542067,0.871155
166,85624211,4243309,0.869939
27,85624211,345075,0.869536
